# EMPIRICAL TRANSFER FUNCTIONS

## Preliminaries

In [2]:
!pip install controlSBML

     |████████████████████████████████| 50 kB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 536 kB 5.2 MB/s eta 0:00:01
     |████████████████████████████████| 122 kB 4.8 MB/s eta 0:00:01
     |████████████████████████████████| 228 kB 3.9 MB/s eta 0:00:01
     |████████████████████████████████| 455 kB 7.4 MB/s eta 0:00:01
     |████████████████████████████████| 8.0 MB 10.7 MB/s eta 0:00:01
     |████████████████████████████████| 101 kB 8.9 MB/s eta 0:00:01
     |████████████████████████████████| 98 kB 6.1 MB/s  eta 0:00:01
     |████████████████████████████████| 840 kB 12.8 MB/s eta 0:00:01
     |████████████████████████████████| 278 kB 6.4 MB/s eta 0:00:01
     |████████████████████████████████| 91 kB 6.6 MB/s  eta 0:00:01
     |████████████████████████████████| 78 kB 2.8 MB/s eta 0:00:01
     |████████████████████████████████| 62 kB 834 kB/s  eta 0:00:01
     |████████████████████████████████| 154 kB 5.3 MB/s eta 0:00:01
     |████████████████████████████████| 4.4 MB 5

In [4]:
import controlSBML as ctl

import control
import lmfit
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tellurium as te
ctl.__version__

ImportError: libncurses.so.5: cannot open shared object file: No such file or directory

## Models

### SIMPLE_MODEL

In [ ]:
SIMPLE_MODEL = """
species E1;
J1: S1 -> S2; S1 + E1
S1 = 10; S2 = 0; E1 = 0;
"""

### LINEAR_MODEL

In [ ]:
LINEAR_MODEL = """
J1: S1 -> S2; k1*S1
J2: S2 -> S3; k2*S2
J3: S3 -> ; k3*S3
S1 = 10; S2 = 0; S3 = 0

k1 = 1
k2 = 1
k3 = 1
"""
rr = te.loada(LINEAR_MODEL)
rr.simulate()
rr.plot()

In [ ]:
LINEAR_CTLSB = ctl.ControlSBML(LINEAR_MODEL, input_names=["S1"], output_names=["S3"])

### WOLF_MODEL

In [ ]:
WOLF_URL = "https://www.ebi.ac.uk/biomodels/model/download/BIOMD0000000206.2?filename=BIOMD0000000206_url.xml"
rr = te.loadSBMLModel(WOLF_URL)
WOLF_MODEL = rr.getAntimony()
WOLF_CTLSB = ctl.ControlSBML(WOLF_MODEL, input_names=["at_"], output_names=["s5"])
print(WOLF_MODEL)

### MTOR_MODEL

In [ ]:
MTOR_URL = "https://www.ebi.ac.uk/biomodels/model/download/BIOMD0000000823.2?filename=Varusai2018.xml"
rr = te.loadSBMLModel(MTOR_URL)
MTOR_MODEL = rr.getAntimony()
MTOR_CTLSB = ctl.ControlSBML(MTOR_MODEL, input_names=["pIR"], output_names=["mTORC1_DEPTOR"])
print(MTOR_MODEL)

# Helpers

In [ ]:
def plotTimeResponse(time_response, output_names,
                     is_legend=True, stmts=None):
    # Plots the results of running a simulation
    outputs = time_response.outputs
    times = time_response.time
    colors = ["orange", "green"]
    for idx in range(len(output_names)):
        if np.ndim(outputs) > 1:
            plt.plot(times, outputs[idx,:], c=colors[idx])
        else:
            plt.plot(times, outputs, c=colors[idx])
    if is_legend:
        _ = plt.legend(output_names)
    if stmts is None:
        stmts = []
    for stmt in stmts:
        exec(stmt)
    plt.xlabel("time")
    plt.ylabel("concentration")

# Linear model transfer function

In this section, we use LINEAR_MODEL to illustrate system identification.
The work can be simplied by making use of the ``SISOTransferFunctionBuilder``
object in ``ControlSBML``.

## Step 1. Construct the ``SISOTransferFunctionBuilder``

In [ ]:
linear_builder = LINEAR_CTLSB.makeSISOTransferFunctionBuilder()
linear_builder.input_name, linear_builder.output_name

## Step 2. Determine the operating region

The operating region of the range of input values that are being considered.
We assess the operating region by using a *staircase input*.
This is a sequence of steps of the same height that have a duration that is sufficiently
long so that the system stabilizes.

In [ ]:
linear_builder.plotStaircaseResponse(initial_value=2, final_value=10, figsize=(3,3),
                                    legend_crd=(0.5, 1), end_time=100)

## Step 3. Find the transfer function

To fit a transfer function, we must specify the number of terms in the
numerator and denominator polynomials.

In [ ]:
fitter_result = linear_builder.fitTransferFunction(1, 2, final_value=10,
                                                   initial_value=2, end_time=100)
ctl.plotOneTS(fitter_result.time_series, figsize=(3,3), legend_crd=(2,1))

This plot displays values of ``S3`` in response to values of ``S1``.
We see that the output closely follows the input, and that there is a response over
the entire range of input.

## Step 4. Analyze the fit result

``fitter_result`` has many useful properties.
* ``minimizer_result`` contains detailed information about the results of the optimization
* ``nfev`` is the number of different transfer functions that were evaluated to find the fit
* ``parameters`` contains the parameter values
* ``redchi`` is the reduced ChiSq for the fit
* ``stderr`` contains the standard deviations of the parameter values
* ``time_series`` is a ``Timeseries`` object with the input, nonlinear simulated output, and predicted value of the output
* ``transfer_function`` is the fitted transfer function

In [ ]:
fitter_result.transfer_function

In [ ]:
fitter_result.nfev

In [ ]:
print(fitter_result.transfer_function)
ctl.plotOneTS(fitter_result.time_series, figsize=(3,3), legend_crd=(2, 1))

#  Wolf Transfer Function

In [ ]:
wolf_builder = WOLF_CTLSB.makeSISOTransferFunctionBuilder()
wolf_builder.input_name, wolf_builder.output_name

In [ ]:
wolf_builder.plotStaircaseResponse(initial_value=2, final_value=10, figsize=(3,3),
                                    legend_crd=(0.5, 1), end_time=100)

In [ ]:
fitter_result = wolf_builder.fitTransferFunction(3, 3, final_value=10,
                                                   initial_value=2, end_time=100)
print(fitter_result.transfer_function)
ctl.plotOneTS(fitter_result.time_series, figsize=(3,3), legend_crd=(2, 1))

In [ ]:
fitter_result.transfer_function.dcgain()

Is this model sufficiently accurate to do control design?

The key consideration here is accuracy of DC Gain. How do the step increases in input relate to the magnitude of the increase in output?

# mTOR transfer function

In [ ]:
mtor_builder = MTOR_CTLSB.makeSISOTransferFunctionBuilder()
mtor_builder.plotStaircaseResponse(initial_value=2, final_value=10, figsize=(3,3),
                                    legend_crd=(2, 1), end_time=2000)

There does not seem to be a relationship between the step input and the output.